In [3]:
# 1. INSTALL DEPENDENCIES
!pip install -q sentence-transformers faiss-cpu datasets langchain>=0.1.0 langchain-community>=0.0.34 langchain-text-splitters

In [ ]:
# 2. IMPORTS
import os 
import json
import zipfile 
import glob
from tqdm import tqdm
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

# 3. LOAD DATA FROM HUGGING FACE
print("=" * 30)
print("LOADING DATASETS FROM HUGGING FACE")

# Dataset 1: OPP-115
print("\nDownloading Dataset 1: OPP-115 (alzoubi36/opp_115)...")
dataset_1 = load_dataset("alzoubi36/opp_115")

# Dataset 2: PrivacyPolicy
print("\nDownloading Dataset 2: PrivacyPolicy (sjsq/PrivacyPolicy)...")
dataset_2 = load_dataset("sjsq/PrivacyPolicy")

# Dataset 3: Kaggle ToSDR
print("\nLoading Dataset 3: Kaggle ToSDR (sonu1607/tosdr-terms-of-service-corpus)...")

# Check for zip file and unzip if needed
zip_files = [f for f in os.listdir('.') if f.endswith('.zip') and ('tos' in f.lower() or 'corpus' in f.lower())]

if zip_files:
    print(f"   Found zip file: {zip_files[0]}")
    print("   Unzipping...")
    try:
        with zipfile.ZipFile(zip_files[0], 'r') as zip_ref:
            zip_ref.extractall('.')
        print("   Unzip complete.")
    except Exception as e:
        print(f"   ❌ Error unzipping: {e}")

# Check for directory containing text files
kaggle_dirs = [d for d in os.listdir('.') if os.path.isdir(d) and ('text' in d.lower() or 'tosdr' in d.lower())]
target_dir = None

# Try to find the directory
for d in kaggle_dirs:
    # Check if it contains txt files
    if any(f.endswith('.txt') for f in os.listdir(d)):
        target_dir = d
        break

dataset_3_docs = []

if target_dir:
    print(f"   Found local directory: {target_dir}")
    txt_files = glob.glob(os.path.join(target_dir, '*.txt'))
    print(f"   Found {len(txt_files)} text files")

    for txt_file in tqdm(txt_files, desc="Loading Kaggle files"):
        try:
            with open(txt_file, 'r', encoding='utf-8', errors='ignore') as f:
                content = f.read()
                if len(content) > 100:
                    doc = Document(
                        page_content=content,
                        metadata={
                            'source': 'kaggle_tosdr',
                            'filename': os.path.basename(txt_file)
                        }
                    )
                    dataset_3_docs.append(doc)
        except Exception as e:
            pass # Skip problematic files

    print(f"   - Kaggle ToSDR: {len(dataset_3_docs)} documents loaded")
else:
    print("   ⚠️  Kaggle dataset directory not found locally.")
    print("   (This is normal if you haven't manually uploaded the Kaggle zip file. We will proceed with the other 2 datasets.)")

print(f"\nDatasets loaded successfully!")
print(f"   - OPP-115 Train: {len(dataset_1['train'])} examples")
print(f"   - PrivacyPolicy Train: {len(dataset_2['train'])} examples")

# 4. CONVERT TO LANGCHAIN DOCUMENTS
print("\n" + "=" * 30)
print("PREPARING DOCUMENTS")

documents = []

# Process Dataset 1 (OPP-115)
print("\nProcessing OPP-115 dataset...")
for split in ['train', 'validation', 'test']:
    if split in dataset_1:
        print(f"  - Processing {split} split...")
        for i, item in enumerate(tqdm(dataset_1[split])):
            doc = Document(
                page_content=item['text'],
                metadata={
                    'source': 'opp_115',
                    'split': split,
                    'index': i,
                    'original_id': f"opp_{split}_{i}"
                }
            )
            documents.append(doc)

# Process Dataset 2 (sjsq/PrivacyPolicy)
print("\nProcessing sjsq/PrivacyPolicy dataset...")
for split in dataset_2.keys():
    print(f"  - Processing {split} split...")
    for i, item in enumerate(tqdm(dataset_2[split])):
        # Normalize column names (this dataset uses 'Text')
        text_content = item.get('Text', item.get('text', ''))
        if text_content and text_content != 'Null':
             doc = Document(
                page_content=text_content,
                metadata={
                    'source': 'sjsq_privacy_policy',
                    'split': split,
                    'index': i,
                    'filename': item.get('filename', 'unknown')
                }
            )
             documents.append(doc)

# Append Kaggle documents
if dataset_3_docs:
    documents.extend(dataset_3_docs)

print(f"\nTotal documents prepared: {len(documents)}")

# 5. CHUNK THE DOCUMENTS
print("\n" + "=" * 30)
print("CHUNKING DOCUMENTS FOR RETRIEVAL")

print("\nSplitting text into searchable chunks...")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ". ", " "],
    length_function=len,
)

chunks = text_splitter.split_documents(documents)

print(f"\n✅ Created {len(chunks)} searchable chunks")
if len(chunks) > 0:
    print(f"   Average chunk size: ~{sum(len(c.page_content) for c in chunks) // len(chunks)} characters")

# 6. CREATE EMBEDDINGS MODEL
print("\n" + "=" * 30)
print("LOADING EMBEDDING MODEL FROM HUGGING FACE")

print("\nLoading: sentence-transformers/all-MiniLM-L6-v2")

# --- IMPORTANT: GPU CHECK ---
import torch
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"   - Using Device: {device.upper()}")
if device == 'cpu':
    print("   ⚠️ WARNING: Running on CPU. This will be slow (15+ mins). Switch runtime to GPU for speed.")

embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2",
    model_kwargs={'device': device},
    encode_kwargs={'normalize_embeddings': True}
)

print("\nEmbedding model loaded successfully!")

# 7. BUILD VECTOR DATABASE
print("\n" + "=" * 30)
print("BUILDING FAISS VECTOR DATABASE")

print("\n🔨 Creating embeddings for all chunks...")
if len(chunks) > 5000 and device == 'cpu':
    print("   (You have many chunks and are using CPU. This might take 10-20 minutes.)")
else:
    print("   (Processing...)")

vectorstore = FAISS.from_documents(chunks, embeddings)

print("\nVector database created successfully!")
print(f"   Total vectors: {vectorstore.index.ntotal}")

# 8. TEST THE DATABASE
print("\n" + "=" * 30)
print("TESTING RETRIEVAL")

test_queries = [
    "Can the company sell my personal data to third parties?",
    "Do they use my content to train AI models?",
    "Can I delete all my data from their servers?",
    "Am I forced into arbitration instead of court?"
]

print("\nRunning test queries...\n")

for query in test_queries:
    print(f"Query: '{query}'")
    results = vectorstore.similarity_search(query, k=2)
    if results:
        print(f"Found {len(results)} relevant chunks")
        print(f"   Top result preview: {results[0].page_content[:150]}...")
    else:
        print("   No results found.")
    print()

# 9. SAVE THE DATABASE
print("=" * 30)
print("SAVING DATABASE")

print("\n💾 Saving FAISS database to disk...")
vectorstore.save_local("faiss_index_tos_hf")

print("Database saved to: faiss_index_tos_hf/")

# 10. CREATE METADATA FILE
metadata = {
    "dataset": "OPP-115 + PrivacyPolicy",
    "total_documents": len(documents),
    "total_chunks": len(chunks),
    "embedding_model": "sentence-transformers/all-MiniLM-L6-v2",
    "embedding_dimension": 384,
    "chunk_size": 800,
    "chunk_overlap": 100,
    "vector_database": "FAISS",
    "date_created": "2024-12",
    "license": "Apache 2.0",
    "use_case": "Terms of Service Analysis - IAT 360"
}

with open('faiss_index_tos_hf/metadata.json', 'w') as f:
    json.dump(metadata, f, indent=2)

print("Metadata saved")

# 11. ZIP FOR DOWNLOAD
print("\nCreating zip file for download...")
!zip -r -q faiss_index_tos_hf.zip faiss_index_tos_hf

print("\n" + "=" * 30)
print("SUCCESS! DATABASE READY FOR DEPLOYMENT")

Installation complete!

LOADING DATASETS FROM HUGGING FACE



Loading Dataset 3: Kaggle ToSDR (sonu1607/tosdr-terms-of-service-corpus)...
   Found zip file: tos.zip
   Unzipping...
   Unzip complete.
   Found local directory: text
   Found 9491 text files


Loading Kaggle files: 100%|██████████| 9491/9491 [00:00<00:00, 16790.97it/s]


   - Kaggle ToSDR: 9488 documents loaded

Datasets loaded successfully!
   - OPP-115 Train: 2185 examples
   - PrivacyPolicy Train: 1789 examples

PREPARING DOCUMENTS

Processing OPP-115 dataset...
  - Processing train split...


100%|██████████| 2185/2185 [00:00<00:00, 24022.17it/s]


  - Processing validation split...


...


Processing sjsq/PrivacyPolicy dataset...
  - Processing train split...


100%|██████████| 1789/1789 [00:00<00:00, 8864.49it/s]



Total documents prepared: 14709

CHUNKING DOCUMENTS FOR RETRIEVAL

Splitting text into searchable chunks...

✅ Created 343120 searchable chunks
   Average chunk size: ~579 characters

LOADING EMBEDDING MODEL FROM HUGGING FACE

Loading: sentence-transformers/all-MiniLM-L6-v2
   - Using Device: CUDA


/tmp/ipython-input-3330005816.py:167: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(



Embedding model loaded successfully!

BUILDING FAISS VECTOR DATABASE

🔨 Creating embeddings for all chunks...
   (Processing...)

Vector database created successfully!
   Total vectors: 343120

TESTING RETRIEVAL

Running test queries...

Query: 'Can the company sell my personal data to third parties?'
Found 2 relevant chunks
   Top result preview: YOUR PERSONAL DATA WILL NOT BE SOLD. Nor will Your Personal Data be licensed or disclosed to unaffiliated third-parties, except in connection with the...

Query: 'Do they use my content to train AI models?'
Found 2 relevant chunks
   Top result preview: 50.2. You and your End Users are responsible for all decisions made, advice given, actions taken, and failures to take action based on your use of AI ...

Query: 'Can I delete all my data from their servers?'
Found 2 relevant chunks
   Top result preview: Delete your data: You also hold the right to have your personal data deleted.
This is sometimes known as the 'right to be forgotten'.
To re